A is consequent and B is antecedent<br>
Support = P(A&B)<br>
Confidence = Support/P(A)<br>
Lift = Confidence/P(B)<br>

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
retail = pd.read_csv('retail_clean.csv', skipinitialspace=True)
for col in retail.columns:
    if retail[col].dtype == 'O': retail[col] = retail[col].str.strip()

In [3]:
retail.head()

,Unnamed: 0,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue
0,0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


In [4]:
order_size = retail.groupby('Invoice')['Description'].count()

In [5]:
market_basket = retail.groupby(['Invoice','Description']).size().to_frame('count').unstack().fillna(0)

In [6]:
market_basket.columns = market_basket.columns.droplevel()

In [7]:
market_basket

Description,10 COLOUR SPACEBOY PEN,11 PC CERAMIC TEA SET POLKADOT,12 ASS ZINC CHRISTMAS DECORATIONS,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 MINI TOADSTOOL PEGS,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
Invoice,,,,,,,,,,,,,,,,,,,,,
489434,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [9]:
def encoding(x):
    if x<=0: return False
    else: return True

In [10]:
market_basket = market_basket.applymap(encoding)

C:\Users\itunu\AppData\Local\Temp\ipykernel_17068\3163349392.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  market_basket = market_basket.applymap(encoding)


In [11]:
frequent_items = apriori(market_basket, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_items, metric='lift')
rules.sort_values('confidence', ascending=False, inplace=True)

In [14]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
354,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.016146,0.025233,0.014415,0.892797,35.381759,0.014008,9.092746,0.987684
348,"(PINK REGENCY TEACUP AND SAUCER, REGENCY CAKES...",(GREEN REGENCY TEACUP AND SAUCER),0.011521,0.025233,0.010223,0.887324,35.164848,0.009932,8.651055,0.982887
296,(POPPY'S PLAYHOUSE LIVINGROOM),(POPPY'S PLAYHOUSE KITCHEN),0.012441,0.016660,0.011034,0.886957,53.238989,0.010827,8.698778,0.993578
352,"(GREEN REGENCY TEACUP AND SAUCER, PINK REGENCY...",(ROSES REGENCY TEACUP AND SAUCER),0.016903,0.028560,0.014415,0.852800,29.860114,0.013932,6.599458,0.983129
293,(POPPY'S PLAYHOUSE BEDROOM),(POPPY'S PLAYHOUSE KITCHEN),0.015037,0.016660,0.012765,0.848921,50.955924,0.012515,6.508775,0.995342
...,...,...,...,...,...,...,...,...,...,...
92,(WHITE HANGING HEART T-LIGHT HOLDER),(GIN + TONIC DIET METAL SIGN),0.132197,0.037458,0.010602,0.080196,2.140983,0.005650,1.046465,0.614108
105,(WHITE HANGING HEART T-LIGHT HOLDER),(HAND OVER THE CHOCOLATE SIGN),0.132197,0.035159,0.010575,0.079992,2.275152,0.005927,1.048731,0.645848
325,(WHITE HANGING HEART T-LIGHT HOLDER),(WOOD S/3 CABINET ANT WHITE FINISH),0.132197,0.027099,0.010439,0.078969,2.914047,0.006857,1.056317,0.756894
223,(WHITE HANGING HEART T-LIGHT HOLDER),(LUNCH BAG CARS BLUE),0.132197,0.048600,0.010358,0.078355,1.612233,0.003934,1.032284,0.437590


Slow moving items

In [13]:
items_move = retail.groupby('Description').agg({'Quantity':'sum'})

In [106]:
items_move['cut'] = pd.qcut(items_move['Quantity'], 10, labels=False)

In [136]:
slower_moving_items = items_move[items_move['cut']<=8].index.unique()

In [137]:
len(slower_moving_items)

4718

In [138]:
rules = rules.explode('antecedents').explode('consequents')

In [139]:
rules[rules['consequents'].isin(slower_moving_items)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
296,POPPY'S PLAYHOUSE LIVINGROOM,POPPY'S PLAYHOUSE KITCHEN,0.012441,0.016660,0.011034,0.886957,53.238989,0.010827,8.698778,0.993578
292,POPPY'S PLAYHOUSE BEDROOM,POPPY'S PLAYHOUSE KITCHEN,0.015037,0.016660,0.012765,0.848921,50.955924,0.012515,6.508775,0.995342
294,POPPY'S PLAYHOUSE LIVINGROOM,POPPY'S PLAYHOUSE BEDROOM,0.012441,0.015037,0.010331,0.830435,55.225407,0.010144,5.808755,0.994262
293,POPPY'S PLAYHOUSE KITCHEN,POPPY'S PLAYHOUSE BEDROOM,0.016660,0.015037,0.012765,0.766234,50.955924,0.012515,4.213452,0.996985
295,POPPY'S PLAYHOUSE BEDROOM,POPPY'S PLAYHOUSE LIVINGROOM,0.015037,0.012441,0.010331,0.687050,55.225407,0.010144,3.155649,0.996883
297,POPPY'S PLAYHOUSE KITCHEN,POPPY'S PLAYHOUSE LIVINGROOM,0.016660,0.012441,0.011034,0.662338,53.238989,0.010827,2.924694,0.997841
82,FELTCRAFT PRINCESS LOLA DOLL,FELTCRAFT PRINCESS CHARLOTTE DOLL,0.016146,0.016795,0.010250,0.634841,37.799100,0.009979,2.692538,0.989521
83,FELTCRAFT PRINCESS CHARLOTTE DOLL,FELTCRAFT PRINCESS LOLA DOLL,0.016795,0.016146,0.010250,0.610306,37.799100,0.009979,2.524683,0.990174
126,RED KITCHEN SCALES,IVORY KITCHEN SCALES,0.021149,0.028830,0.011765,0.556266,19.294498,0.011155,2.188630,0.968658
42,PINK HAPPY BIRTHDAY BUNTING,BLUE HAPPY BIRTHDAY BUNTING,0.021907,0.017120,0.011819,0.539506,31.513808,0.011444,2.134405,0.989955
